In [1]:
%autosave 0
import requests
import io

from datetime import date, timedelta

## scrap data from source
url="https://covid.ourworldindata.org/data/ecdc/full_data.csv"
s=requests.get(url).content
c=pd.read_csv(io.StringIO(s.decode('utf-8')))
c.head(3)

Autosave disabled


,date,location,new_cases,new_deaths,total_cases,total_deaths
0,2019-12-31,Afghanistan,0,0,0,0
1,2020-01-01,Afghanistan,0,0,0,0
2,2020-01-02,Afghanistan,0,0,0,0


In [2]:
c.tail(3)

,date,location,new_cases,new_deaths,total_cases,total_deaths
16661,2020-05-10,Zimbabwe,1,0,36,4
16662,2020-05-11,Zimbabwe,0,0,36,4
16663,2020-05-12,Zimbabwe,1,0,37,4


In [3]:
## dump the file for using it in other nb
c.to_csv(f"AzureSC\\Base_Files\\full_data{date.today()}.csv", index=False)

In [4]:
## Load data from days -1 to evaluate the performances of model
data_load = pd.read_csv(f"AzureSC\\Pred\\predDf{date.today() - timedelta(days=1)}.csv", index_col=0)
data_load

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-05-11,France,139466.0,0.0,26478.0,0.0,0.0,0.0
0,2020-05-11,China,84098.0,0.0,4660.0,0.0,0.0,0.0
0,2020-05-11,Italy,220169.0,0.0,30754.0,0.0,0.0,0.0
0,2020-05-11,Spain,225035.0,0.0,26829.0,0.0,0.0,0.0
0,2020-05-11,United States,1356465.0,0.0,80864.0,0.0,0.0,0.0
0,2020-05-11,World,4157379.0,0.0,286860.0,0.0,0.0,0.0
0,2020-05-11,United Kingdom,223436.0,0.0,32133.0,0.0,0.0,0.0
0,2020-05-11,Germany,170236.0,0.0,7435.0,0.0,0.0,0.0
0,2020-05-11,Iran,109209.0,0.0,6696.0,0.0,0.0,0.0
0,2020-05-11,Turkey,140036.0,0.0,3828.0,0.0,0.0,0.0


In [5]:
def Eval(c,data_load,country):
    """ Factorise function for evaluate all models
    params : 
        c = data scrap today
        data_load = dataset from day -1
        country = country concerned by eval
    """
    df = c
    df = df[df['location'].isin([f"{country}"])]
    sle = df.iloc[-1]
    v0 = sle["date"]
    v1 = sle["total_cases"]
    v2 = sle["total_deaths"]
    
    res_tempo = data_load[data_load['country'].isin([f"{country}"])]
    rez1 = res_tempo.loc[0,"total_cases_predict"]
    rez2 = res_tempo.loc[0,"total_deaths_predict"]
    
    errorCase1 = rez1-v1
    errorCase2 = rez2-v2

    follow_df = pd.DataFrame()
    follow_df.loc[0,"date"] = v0
    follow_df.loc[0,"country"] = country
    follow_df.loc[0,"total_cases_predict"] = rez1
    follow_df.loc[0,"total_cases_real"] = v1
    follow_df.loc[0,"total_deaths_predict"] = rez2
    follow_df.loc[0,"total_deaths_real"] = v2
    follow_df.loc[0,"error_abs_cases"] = errorCase1
    follow_df.loc[0,"error_abs_deaths"] = errorCase2
    
    return follow_df

In [6]:
## call eval function and store dataframe on var's
follow_df1 = Eval(c,data_load,"France")
follow_df2 = Eval(c,data_load,"China")
follow_df3 = Eval(c,data_load,"Italy")
follow_df4 = Eval(c,data_load,"Spain")
follow_df5 = Eval(c,data_load,"United States")
follow_df6 = Eval(c,data_load,"World")
follow_df7 = Eval(c,data_load,"United Kingdom")
follow_df8 = Eval(c,data_load,"Germany")
follow_df9 = Eval(c,data_load,"Iran")
follow_df10 = Eval(c,data_load,"Turkey")

In [7]:
## concatenate all df in 1
frames = [follow_df1,follow_df2,follow_df3,follow_df4,follow_df5,follow_df6,follow_df7,follow_df8,follow_df9,follow_df10]
rapport = pd.concat(frames)
rapport

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-05-12,France,139466.0,139519.0,26478.0,26643.0,-53.0,-165.0
0,2020-05-12,China,84098.0,84011.0,4660.0,4637.0,87.0,23.0
0,2020-05-12,Italy,220169.0,219814.0,30754.0,30739.0,355.0,15.0
0,2020-05-11,Spain,225035.0,227436.0,26829.0,26744.0,-2401.0,85.0
0,2020-05-12,United States,1356465.0,1347916.0,80864.0,80684.0,8549.0,180.0
0,2020-05-12,World,4157379.0,4137193.0,286860.0,285760.0,20186.0,1100.0
0,2020-05-12,United Kingdom,223436.0,223060.0,32133.0,32065.0,376.0,68.0
0,2020-05-12,Germany,170236.0,170508.0,7435.0,7533.0,-272.0,-98.0
0,2020-05-12,Iran,109209.0,109286.0,6696.0,6685.0,-77.0,11.0
0,2020-05-12,Turkey,140036.0,139771.0,3828.0,3841.0,265.0,-13.0


In [8]:
## dump dataset for use it in other nb
rapport.to_csv(f"AzureSC\\Rapport\\rap{date.today()}.csv")